# MODELO 1º CLUSTERIZACIÓN FRAGMENTADA

## Objetivo

#### Este primer modelo tiene como objetivo clusterizar los puntos de reparto de mi base de datos para optimizar las rutas de transporte. Para ello me valdré de las siguientes librerías, algunas de las cuales no tendrán repercusión en el modelo. He querido mantener en cierta medida los pasos que he ido dando en el desarrollo del modelo para comunicar mis impresiones, así como las complicaciones que me he ido encontrando, ya que, ante todo, este proyecto tiene una finalidad didáctica


- Librerias utilizadas para el desarrollo efectivo del modelo

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import math

#libreria googlemaps para python:
#! pip install -U googlemaps
import googlemaps

#Actualizar el pip instalador
#! pip install --upgrade pip

%matplotlib inline

- Librerías usadas pero no utilizadadas para el desarrollo efectivo del modelo 

##### Libreria geopy: geocodificar direcciones:

! pip install geopy

from geopy.geocoders import Nominatim

**GEOPY**: Antes de empezar a trabajar con la plataforma de Google Maps, investigo acerca de esta librería de GEOPY, la cual geocodifica direcciones. El objetivo era aprender alguna manera alternativa a Google para geocodificar, ya que necesitaría sacar dos tipos de productos de Google (distance matrix y geocoding), lo cual tiene un coste. Tras varios intentos parece que funciona, aunque bastante regular dando ubicaciones sin sentido (ej:calle orense 32, Madrid)

Nota: en el 2º Modelo veremos una tercera vía para geocodificar (GEOPANDAS) aunque también limita el número de peticiones

###### ejemplo GEOPY
- geolocator = Nominatim(user_agent="user_project")
- location = geolocator.geocode("Calle del Universo, 3, Valladolid")
- print((location.latitude, location.longitude))
(41.6569033, -4.6990056)
- print(location.raw)
{'place_id': '93034279', 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': '75689481', 'boundingbox': ['41.6550687', '41.6587924', '-4.6990298', '-4.698207'], 'lat': '41.6569033', 'lon': '-4.6990056', 'display_name': 'Calle del Universo, Pilarica, Valladolid, Castilla y León, 47011, España', 'class': 'highway', 'type': 'residential', 'importance': 0.8099999999999999}

### INICIO: LECTURA Y SELECCIÓN DE LOS DATOS

Los datos vienen en formato excel que es el que suelen usar en la empresa de transporte. La información que nos interesa es la relativa a la dirección (dirección,CP,Población) y fecha

In [ ]:
df_max = pd.read_excel('data_prueba.xls', usecols=[5, 6,7,15], convert_float=True, skip_footer=2, parse_dates=['FECHA REPARTO'])

###### Creo una columna unificando las 3 columnas relativas a dirección para mejor entendimiento de la API de Google

In [ ]:
df_max['DESTINO'] = df_max[['DIRECCION','CP DESTINO','POBLACION DESTINO']].apply(lambda x : '{}, {}, {}'.format(x[0],x[1],x[2]), axis=1)

In [ ]:
df_max.info()

La salida es un dataframe sobre el que trabajaremos para cumplir el objetivo

In [ ]:
# FILTRADO POR FECHA DE REPARTO. En este caso el 04/02/2019 es la que presenta el grueso de repartos
date_of_interest = pd.datetime(2019,2,4)
df_max = df_max[ df_max['FECHA REPARTO']==date_of_interest]
df_max.shape

**REMOVIDO DE DUPLICADOS**: En la base de datos es muy común que haya más de una entrega en la misma dirección, bien por consistir en diferente tipo de entrega (paletizado/mensajería) o simplementa que coincida el destino. Como el objetivo es optimizar la ruta, lo que haremos será quedarnos con los destinos borrando los duplicados. Así mismo se pasará al jefe de trafico los destinos duplicados para que lo tenga en cuenta en la carga de mercancía

In [ ]:
# DUPLICADOS EN LAS ENTREGAS:

same_directions = np.array([],dtype=str)#datos a sacar por pantalla al jefe de tráfico en Tableau
differents_directions = np.array([],dtype=str)

for i in df_max['DESTINO']:            
        if i not in differents_directions:
                differents_directions = np.append(differents_directions,i)
        else:
            same_directions = np.append(same_directions,i)  
            #print('ten en cuenta que la dirección %s tiene más de una entrega' % same_direction)
            
# REMOVIDO DE ENTREGAS DUPLICADAS EN EL MISMO DESTINO PARA LIMPIAR EL DATAFRAME
df_max.drop_duplicates(inplace=True)
df_max.reset_index(drop=True,inplace=True)
len(df_max)

## GOOGLE MAPS PLATFORM

**LOGGIN**: obtengo una cuenta y un proyecto con Google Maps, lo cual me permitirá aprovechar sus utilidades. Hay que epsecificar los servicios que se requeriran ya que son llamadas de API diferenciadas

### Instrucción al examinador: la API Key la guardo en un documento .py la cual importo, este documento se envía por correo electrónico

In [ ]:
from info_tfm import api_key
api_key

In [ ]:
user_google = googlemaps.Client(key=api_key)
user_google

In [ ]:
user_google

#### Accedo a un REPOSITORIO abierto de desarrolladores de google maps para PYTHON, la cual me brinda la posibilidad de poder trabajar desde Python y acceder a los direferentes servicios Google Maps Platform tiene. 

Repositorio con la documentación: https://googlemaps.github.io/google-maps-services-python/docs/#   

# DISTANCE MATRIX 

#### El primer objetivo es crear la matriz de distancias entre los diferentes destinos de entrega. Google aporta una serie de servicios enre los que se incluye la llamada "distance matrix" (con una llamada a la API especifica para ello). Esta es la que usaré a continuación.  En este servicio Google geocodifica por defecto para poder sacar los siguientes parámetros:

- Distancia
- Duración
- Duración en tráfico (si el transporte es vehiculo a motor)
- Status

In [ ]:
from googlemaps import distance_matrix

# Tengo que hacer chunks de reiteración a la API de Google

##### Dado que la API de Google no tolera más de 10 peticiones de combinaciones de origen y destino, y no tiene limite de peticiones diarias, voy a hacer iteraciones de 10 en 10 peticiones

Condiciones Google:

- Limited to 100 elements per client-side request.
- Maximum of 25 origins and 25 destinations per server-side request.
- 1,000 server-side elements per second. *Note that the client-side service offers Unlimited elements per second, per project.

Para más información: https://developers.google.com/maps/documentation/javascript/distancematrix

#### PARÁMETROS DE LA DISTANCE MATRIX DE GOOGLE
- Mode=driving ---> porque nuestros repartos son en vehiculo en su mayor parte
- Departure_time = 'now'---> Debido a que la orden de repartos es en la mañana
- traffic_model = 'pessimistic'---> para no penalizar al conductor si la ruta sufre más atascos de lo normal
- region = '.es'---> usamos la ccTLD (códigos de pais) de España para prevenir duplicidades con direcciones de paises hispanos

## Problema de la clusterización

Se presenta un problema y es que no se puede sacar una matriz de distancias general para todos lo puntos de reparto si superan los 10 destinos, ya que, debido a esta limitación de Google, no se podrán conectar todos los puntos de reparto entre ellos, quedando una matriz de distancias incompleta.

***NOTA***: en el segundo modelo solventaré esta situación y conseguiré una matriz de distancias completa.

#### Es por ello que, para tratar de darle continuidad a esta limitación inicial, trataré cada grupo de 10 combinaciones de entregas como un cluster cerrado. Al final quedarán n clusteres de máximo 10 puntos de reparto (destinos) cada uno. A este paso lo llamaré clusterización de 1º grado

**Solución**: Para crear un objeto con todos los clusteres que devuelve google (de máximo 10 destinos cada) iteraremos la función de matriz de distancias por bloques de 10 destinos correlativamente sobre la columna de DESTINO

### VARIABLE Nº DESTINOS

La variable "n" tiene en cuenta el número de destinos a tomar en cuenta en el modelo. En mi caso cogeré 60 para ambos modelos por cuestiones económicas y de memoria

In [ ]:
df= df_max.head(n=60)

Comunicar que nuestro modelo presenta un coste mayor al usual de petición a Google ya que el parámetro **traffic_model** eleva el precio de petición a Google

# CLUSTERIZACIÓN 1º GRADO

In [ ]:
#origenes y destinos con corte maximo de 10

interval = 10
c_size = 1
dict_clusters={}

for i in range(math.ceil(len(df)/interval)):# importo math para tener la función ceil que redondea al alza
        
        df_chunk = df.loc[c_size:interval]
        dm = distance_matrix.distance_matrix(client=user_google, origins = df_chunk['DESTINO'],
                                     destinations = df_chunk['DESTINO'], mode = 'driving',
                                     departure_time = 'now', traffic_model = 'pessimistic', 
                                     region = '.es')
        dict_clusters[i] = dm
        c_size+= 10
        interval+=10 

### Parámetros de salida

- Valor distance: magnitud en metros
- Valor duration: magnitud en segundos

In [ ]:
# En el bucle anterior no me permite pasarlo a lista directamente, de ahí que se separe
list_clusters = []

for i in dict_clusters.keys():
    
    list_clusters.append(dict_clusters[i])    

In [ ]:
len(list_clusters)

## DESANIDAMIENTO DE LA INFORMACIÓN DE GOOGLE

En este punto tenemos una lista de clusters con el formato sacado directamente de Google. Ese formato presenta una estructura de listas y diccionarios superspuestos de donde tendré que sacar la información para crear mi Dataframe con un formato más homogéneo.

Google devuelve las siguientes Keys de donde parten los datos:

- origin_addresses
- destination_addresses
- rows: dentro viene la información relativa a tiempo y duración, así como duración de tráfico y status
- status

**NOTA**: CUANDO EL STATUS ES *NOT FOUND* NO HAY SALIDA DE LOS PARAMETROS, POR LO QUE NO COINCIDEN LAS LONGITUDES DE LAS SERIES QUE CONFORMAN EL DATAFRAME. POR ELLO CREO UN DATA FRAME INICIAL AL QUE QUITAMOS LAS LINEAS "*NOT FOUND*" PARA QUE LUEGO PUEDA PASAR LOS OTROS PARÁMETROS (DISTANCE, DURATION Y DURATION IN TRAFFIC) Y SE COLOQUEN EN EL ORDEN DESEADO

In [ ]:
# DESANIDAMIENTO

clusters_df = []

for cluster in range(len(list_clusters)):

    origen=[]

    for key,values in list(list_clusters[cluster].items()):    
        if key=='origin_addresses': 
            for i in values:
                for n in range(len(values)):            
                    s1=origen.append(i)

    for i in list_clusters[cluster].keys():    
        if i=='destination_addresses':
            destino = list_clusters[cluster][i]
            destino = destino * len(list_clusters[cluster][i])
            
    distance=[]
    duration=[]
    duration_in_traffic=[]
    status=[]

    for d in list_clusters[cluster].keys():    
        if d=='rows':        
            x2= list_clusters[cluster][d]

    # 1) en esta fase se hace diccionario de la lista previa con los 3 elementos
    for l in x2:

        # 2) en esta fase se hace lista de los valores del diccionario anterior 
            for l1 in l.values():

                # 3) en esta fase se hace diccionario de las partes de la lista anterior   
                for l2 in l1:
                  
                    for key,values in list(l2.items()):   

                        if key=='distance':
                            for key,values in list(values.items()):
                                if key=='value':
                                    distance.append(values)
                        if key=='duration':
                            for key,values in list(values.items()):
                                if key=='value':
                                    duration.append(values) 
                        if key=='duration_in_traffic':
                            for key,values in list(values.items()):
                                if key=='value':
                                    duration_in_traffic.append(values)
                        if key=='status':
                            status.append(values)
                            
    #Creación de cluster en Data Frame
    dfdata = { 'origen' : origen,
               'destino' : destino,
               'status': status
               }
        
    ddm = pd.DataFrame(dfdata,columns=['origen','destino','status'])
    ddm = ddm[ ddm['status']!='NOT_FOUND']# Para evitar nulos y posibilitar que la extensión de las arrays coincidan 
    ddm['distance'] = distance
    ddm['duration'] = duration
    ddm['duration_in_traffic'] = duration_in_traffic
    clusters_df.append(ddm)

**NOTA**: se comprueba que en algunas ocasiones (muy pocas realmente) Google a veces no devuelve el parámetro **duration in traffic**, el cual es el que queremos tener en cuenta en el modelo. Por ello y para que no de error el modelo nos fijamos en el parámetro **duration** (por si acaso Google falla). Es por ello que en el modelo 2 capo la posibildiad de crear el dataframe con "duration in traffic" ya que a veces daba error en la longitud de las arrays de salida. 

**En este modelo 1º no mantengo capada la salida de duration in traffic porque relamente no será nuestro modelo, así que trabajo sobre duration in traffic para que sea vea otra manera alternativa de recopilación de datos**

# OPTIMIZACIÓN INSIDE CLUSTER

###### El siguiente paso es optimizar las rutas dentro de cada cluster.

Tras probar AgglomerativeClustering y scipy.cluster.hierarchy tomo una de las decisiones más importantes de mi TFM y es la de intentar crear mi propio algoritmo de optimización y clusterización de las rutas en base a las variables que estimemos (duration, duration in traffic y distance).

**CLUSTERIZACIÓN**: Como observé en el transcurso del proyecto, dado el tipo de limitación de Google y la tipología de la finalidad buscada (optimizar la ruta), los algoritmos usuales de clusterización no se ajustaban a lo que pretendía. Lo que necesito es un linkage tipo ***single*** que tome en consideración que el punto de partida desde donde medir la distancia al punto más cercano es el último destino. Esto es debido a que en la realidad un repartidor llega a un destino y sólo puede partir de ahí para el siguiente punto más cercano, por lo que ni los modelos ward, complete, los tipo de promedio o incluso el single básico no se ajustarán a la realidad de las rutas.

Entre este motivo y que tras pasar la distance matrix ordenada (y no simétrica), no conseguía estructurar la salida de los clusters usando los algoritmos hierarchy (relativo a origenes y destinos) tomo la decisión de crear mi código.


In [ ]:
# CÓDIGO TABLA OPTIMIZACIÓN INTERNA DE LA LISTA DE CLUSTERES

def optimizer_cluster(clusters_df,var_optimizer='duration'):
    
    # variables iniciales
    index_origin = clusters_df[var_optimizer].idxmin()
    destination = clusters_df.loc[index_origin][0]
    df_exit = pd.DataFrame(columns=('origen','destino','distance','duration','duration_in_traffic'))

    while len(clusters_df)>0:
   
            index = clusters_df[ clusters_df['origen']==destination][var_optimizer].idxmin()
            df_exit = df_exit.append((clusters_df.loc[index]).T)
            origin = clusters_df.loc[index][0]
            destination = clusters_df.loc[index][1]
            clusters_df.drop(index,inplace=True) 
            clusters_df.drop(clusters_df  [ (clusters_df['origen']==destination) & (clusters_df['destino']==origin)].index,inplace=True)
            clusters_df.drop(clusters_df  [  clusters_df['destino']==destination].index,inplace=True)
            clusters_df.drop(clusters_df  [  clusters_df['destino']==origin].index,inplace=True)
        
    return df_exit

In [ ]:
# Paso intermedio de removido de distancias nulas y de la columna status (inservible ya).
for i in range(len(clusters_df)):
        clusters_df[i] = clusters_df[i] [(clusters_df[i]['distance']!=0)]
        clusters_df[i].reset_index(drop=True,inplace=True)
        clusters_df[i].drop('status',axis=1,inplace=True)

In [ ]:
# Creamos una copia para darle continuidad a partir de aquí (para gestionar posibles fallos que se daban y no repetir el código)
clusters_df_optimazed = clusters_df.copy()

In [ ]:
# LE PASAMOS LA FUNCIÓN OPTIMIZER A CADA CLUSTER PARA CONOCER LA RUTA MÁS OPTIMA
clusters_df_optimazed = list(map(optimizer_cluster,(clusters_df_optimazed)))
clusters_df_optimazed

In [ ]:
clusters_df_optimazed[4]

# CLUSTERIZACIÓN 2º GRADO

##### Recapitulación: hasta ahora lo que hemos hecho es crear clusters de máximo 10 entregas cada uno (n<=10), obteniendo una lista de clusters a la que pasamos la función optimizer. Al pasarle esta función lo que hacemos es imitar el algoritmo de clusterización hierarchycal clustering, usando el equivalente al linkage "single" pero adecuado a realidad de un repartidor, en la que el punto de partida es siempre el de la última entrega. 

### Lo que vamos a hacer ahora es intentar conectar los origenes y destinos de estos clusteres ya creados para seguir agrupando de la manera más eficiente

Cada cluster ya calculado (clusterización de 1º grado) se conectará con el resto de clusters. Para ello, el destino de un cluster de 1º grado (inside_cluster) sería el punto de partida (origen) desde el que conectar con otro cluster de 1º grado. Asímismo, los origenes de los clusteres de 1º grado serían los posibles destinos a los que conectar. Es por ello que, de forma resumida, se invierten los puntos origenes y destinos de la clusterización de 1º grado, pasando estos a ser destinos y origenes en su conexión al resto de clusteres (clusterización de 2º grado/outside_cluster). 

##### Resalto que esta parte puede resultar algo complicada de entender porque lo que antes era último destino inside_cluster ahora se convierte en origen outside_cluster y viceversa. Usaremos ambas definiciones según estemos inside_cluster u outside_cluster

In [ ]:
# EN ESTE PASO INVIERTO LOS ORIGENES,DESTINOS DENTRO DE CADA CLUSTER CREANDO UN DF PARA CONOCER LAS CONEXIONES 2º GRADO

origenes = {} #los usaremos  para dropear luego
destinos = {}

for cluster in range(len(clusters_df_optimazed)):
    
    for i in list(clusters_df_optimazed[cluster]['destino'].tail(1)):
        origenes[cluster] = i

    for i in list(clusters_df_optimazed[cluster]['origen'].head(1)):
        destinos[cluster] = i

    
dfdata = { 'origen' : origenes,
               'destino' : destinos,
               }

df_ori_des = pd.DataFrame(dfdata,columns=['origen','destino'],dtype=str)
df_ori_des

#### DICCIONARIOS ÚTILES PARA ULTERIORES FUNCIONES

Creamos diccionarios de origen y destino **"inside cluster"** y **"outsidecluster"** para ver las consexiones.

Peligro: No pueden coincidir rows de origenes y destinos porque sino el diccionario descarta el duplicado (ya que actúa como un set). No debería haber duplicados porque los hemos quitado, pero Google puede crear duplicados al geocodificarlos (resumiendo destinos a Codigo Postal y población).

In [ ]:
ori_des_outsidecluster = dict(zip(origenes.values(),destinos.values()))
ori_des_insidecluster = dict(zip(destinos.values(),origenes.values()))
ori_des_outsidecluster,ori_des_insidecluster

In [ ]:
# EL SIGUIENTE PASO ES OBTENER LA DISTANCE MATRIX ENTRE LOS DIFERENTES CLUSTERES PARA OBTENER LA MEJOR AGRUPACIÓN DE ELLOS

dm_clusters = distance_matrix.distance_matrix(client=user_google, origins = df_ori_des['origen'],
                                     destinations = df_ori_des['destino'], mode = 'driving',
                                     departure_time = 'now', traffic_model = 'pessimistic', 
                                     region = '.es')

### DESANIDAMIENTO Y OPTIMIZACIÓN CLUSTERIZACIÓN 2º GRADO

Similar al paso de la 1º Clusterización pero pero el objetivo es diferente. En este caso queremos crear un dataframe que me genere las pautas de conexión entre los clusteres iniciales, para, poseteriomente (como veremos), agruparlos optimizando según las variables que estimemos (distance, duration, duration in traffic)

El código de desanidamiento varía con el anterior ya que en este no iteramos sobre más clusteres. No he querido ampliar el modelo a más de 100 destinos (en dado caso, debido a la limitación de Google debería crear lista de clusters como en la clusterización de 1º Grado) para cerrar antes este modelo. No me aportaba más creatividad y sí mucha codificación repetida, a la par que no es el modelo escogido.

In [ ]:
# Transformación del código en data frames

origen=[]

for key,values in list(dm_clusters.items()):    
    if key=='origin_addresses': 
        for i in values:
            for n in range(len(values)):            
                s1=origen.append(i)

for i in dm_clusters.keys():    
    if i=='destination_addresses':
        destino = dm_clusters[i]
        destino = destino * len(dm_clusters[i])

distance=[]
duration=[]
duration_in_traffic=[]
status=[]

for d in dm_clusters.keys():    
    if d=='rows':        
        x2= dm_clusters[d]

# 1) en esta fase se hace diccionario de la lista previa con los 3 elementos
for l in x2:

    # 2) en esta fase se hace lista de los valores del diccionario anterior 
        for l1 in l.values():

            # en esta fase se hace diccionario de las partes de la lista anterior   
            for l2 in l1:


                for key,values in list(l2.items()):   

                    if key=='distance':
                        for key,values in list(values.items()):
                            if key=='value':
                                distance.append(values)
                    if key=='duration':
                        for key,values in list(values.items()):
                            if key=='value':
                                duration.append(values) 
                    if key=='duration_in_traffic':
                        for key,values in list(values.items()):
                            if key=='value':
                                duration_in_traffic.append(values)

                    if key=='status':
                        status.append(values)

#Creación de cluster en Data Frame
dfdata_clusters = { 'origen' : origen,
           'destino' : destino,
           'status': status
           }

ddm_clusters = pd.DataFrame(dfdata_clusters,columns=['origen','destino','status'])
ddm_clusters = ddm_clusters[ ddm_clusters['status']!='NOT_FOUND']# Para evitar nulos y posibilitar 
#que la extensión de las arrays coincidan. En este caso es insospechado que pueda haber NOT FOUNDS
ddm_clusters['distance'] = distance
ddm_clusters['duration'] = duration
ddm_clusters['duration_in_traffic'] = duration_in_traffic
#exec(df_cluster{i} = ddm).format(i)
ddm_clusters

In [ ]:
ddm_clusters.drop('status',axis=1,inplace=True)

Como observamos, este dataframe de distance matrix también cruza los origenes y destinos propios de cada cluster de 1º grado (inside_cluster). Estos los tenemos que quitar para no tenerlos en cuenta en la optimización posterior ya que un cluster de 1º grado no se va a conectar a él mismo. Es lo que hacemos en el código posterior

In [ ]:
# Los origenes y destinos internos de cada cluster no tienen que tenerse en cuenta para el linkage

for cluster in range(len(df_ori_des)):
        print(origenes[cluster])
        ddm_clusters.drop(ddm_clusters  [ (ddm_clusters['origen']==origenes[cluster])
                                     & (ddm_clusters['destino']==destinos[cluster])].index,inplace=True)               


In [ ]:
ddm_clusters

In [ ]:
#Creamos una copia para darle continuidad a partir de aquí (para gestionar posibles fallos que se daban y no repetir el código)
ddm_clusters_optimized = ddm_clusters.copy()

#### OPTIMIZACIÓN

Optimización de esta **clusterización de 2º Grado** para conectar los clusteres de 1º grado. Lo que se pretende es conectar los cluteres así que lo que hacemos es hacerlo por el linkage single de menor distancia entre los origenes y destinos

Se diferencia del primero en la estructura de origenes y destinos por lo que **no** se hace con la **función optimizer**


**NOTA:** Tanto en este paso (como en el similar del 1º grado) se vacía el dataframe que sirve de base, lo que tendrá consecuencias de no retroceso en el proceso. Esto es porque lo que hacemos es dropear las lineas que ya se han incluido en el nuevo dataframe optimizado (a la par que dropeamos las conexiones que ya no se darán)

In [ ]:

index_origin = ddm_clusters_optimized['duration_in_traffic'].idxmin()
destination = ddm_clusters_optimized.loc[index_origin][0]
df_exit = pd.DataFrame(columns=('origen','destino','distance','duration','duration_in_traffic'))

while len(ddm_clusters_optimized)>0:
    
    index = ddm_clusters_optimized[ ddm_clusters_optimized['origen']==destination]['duration_in_traffic'].idxmin()
    df_exit = df_exit.append((ddm_clusters_optimized.loc[index]).T)
    destination_initial = ddm_clusters_optimized.loc[index][1]
    origin_initial = ddm_clusters_optimized.loc[index][0]
    ddm_clusters_optimized.drop(ddm_clusters_optimized  [  ddm_clusters_optimized['destino']==destination_initial].index,inplace=True)
    ddm_clusters_optimized.drop(ddm_clusters_optimized  [  ddm_clusters_optimized['origen']==origin_initial].index,inplace=True)
    ddm_clusters_optimized.drop(ddm_clusters_optimized  [  ddm_clusters_optimized['destino']==ori_des_outsidecluster[origin_initial]].index,inplace=True)
    destination = ori_des_insidecluster[destination_initial]
    
df_exit

Realmente lo que nos interesa es el orden de conexión de los clusteres de 1º Grado, por lo que en el siguiente código sacamos una lista con el orden de recorrido de los clusteres (tras optimizar sus conexiones)

In [ ]:
# Creo una lista con el orden de origen-destinos necesario para ordenar posteriormente los clusters
df_intercluster_exit = []

for i in range(len(df_exit)):
    if i ==0:
        df_intercluster_exit.append(df_exit.iloc[0]['origen'])
    df_intercluster_exit.append(df_exit.iloc[i]['destino'])

df_intercluster_exit

Siguiendo con lo indicado, saco el orden de conexión de los clusteres de 1º grado

In [ ]:
#ORGANIZACIÓN DE LOS CLUSTERES

orden_clusteres_list = []

for i in range(len(df_intercluster_exit)):
    if i==0:
        orden_clusteres_list.append(list(df_ori_des [df_ori_des['origen'] == df_intercluster_exit[i]].index))
    else:
        orden_clusteres_list.append(list(df_ori_des [df_ori_des['destino'] == df_intercluster_exit[i]].index))
        
orden_clusteres_int = []

for order in orden_clusteres_list:
    for i in order:
        orden_clusteres_int.append(i)
        
orden_clusteres_int

## PREPARACIÓN DATAFRAME FINAL

##### Reorganización de los clusters de 1º grado en función del resultado de la clusterización de 2º grado

In [ ]:
total_clusters_optimazed = pd.DataFrame()

for order in orden_clusteres_int:
    total_clusters_optimazed = total_clusters_optimazed.append(clusters_df_optimazed[order])

#Reseteo para quitarme los index antiguos de la llamada a la API.
total_clusters_optimazed.reset_index(drop=True)

total_clusters_optimazed

# FASE CLUSTERIZACIÓN 3º GRADO

En este momento disponemos de un conjunto ordenado por la variable elegida (duration en nuestro caso) optimizadas anteriormente. La clusterización  de 3º grado consiste en reorganizar los clusters atendiendo al tiempo de reparto de los conductores. Es por ello que segmentaremos el dataframe en clusteres atendiendo a la variable **tiempo en ruta**

#### Técnicamente, lo que hemos hecho en los procesos anteriores es clusterizar sin seguir un criterio. Sólo nos limitábamos a a seguir las limitaciones de Google para luego solventar esta limitación en el proceso de 2º Clusterización

Lo que nos proponemos ahora es fijar nuestro criterio para clusterizar. Este criterio o variable limitará el número de destinos o rutas por cluster en función de que se cumpla el argumento que queramos. Esta variable es el **tiempo en ruta** que queremos asignar a cada reaprtidor para que se ajuste a su horario

Esta fase es la clave para comprender la finalidad de nuestro modelo,  que tiene tanto una vertiente económica relacionada con la optimización del trabajo de cada repartidor, como decalidad del servicio otorgado

#### Es por ello que la métrica comparativa entre modelos será el tiempo estimado de las rutas totales

### CÓDIGO DE CLUSTERIZACIÓN ATENDIENDO AL CRITERIO ESPECIFICADO

Creo la función **my_cumsum_func** que genera clusters nuevos en el momento en el que la duración de la ruta supera el tiempo fijado de reparto de nuestros conductores

En esta parte ya empiezo a generar arrays de numpy

In [ ]:
def my_cumsum_func(column_duration,time_route=21600):
        grp = np.zeros(len(column_duration))
        grp[0] = 0
        #dfdata = { 'cumsum_duration' : column_duration,'clusters': grp }

        for i in range(1,len(column_duration)):

            if (column_duration[i-1] + column_duration[i]) <= time_route:
                grp[i] = grp[i-1]
                column_duration[i] = column_duration[i-1] + column_duration[i]
            else:
                grp[i] = grp[i-1] + 1
                
        result = np.array([column_duration,grp])

        return result

In [ ]:
#para reiniciar la prueba del orden de clusteres
column_duration = np.array(total_clusters_optimazed['duration_in_traffic'])
len(column_duration)

In [ ]:
#APLICO LA FUNCIÓN CUMSUM SOBRE LA DURACIÓN EN TRÁFICO
array_cumsum = my_cumsum_func(column_duration)

In [ ]:
array_cumsum

In [ ]:
#AUMENTO DEL DATAFRAME TOTAL CON LOS CLUSTERES Y EL CUMSUM
total_clusters_optimazed['cumsum_duration']=array_cumsum[0]
total_clusters_optimazed['clusters']=array_cumsum[1]

In [ ]:
len(total_clusters_optimazed)

In [ ]:
total_clusters_optimazed

In [ ]:
total_clusters_optimazed['duration'].sum()

# ANALISIS DE MÉTRICAS

### El apartado del análisis de las metricas entre los dos modelos radicará en comparar la variable *duration*. Aquel modelo que mejor se comporte será el que implementemos la puesta en marcha para la visualización

Hipotesis: la prueba comparativa tendrá en cuenta para ambos modelos:

- Variable escogida: duration (ya que duration en traffic hemos comprobado que a veces Google no devuelve el valor)
- Nº de repartos/destino = 60 (para ambos modelos)

1º Modelo: total_clusters_optimazed['duration'].sum() = 26440

2º Modelo: df_exit['duration'].sum() = 17963

Como era de esperar, se reduce más el tiempo en el segundo modelo, ya que pone en relación directa a todos los destinos, pudiendo conformar una DISTANCE MATRIX unica para todos.
En detrimento de esta optimización del tiempo se puede ver un peor comprotamiento del segundo modelo relativo al tiempo de resuesta de la API de Google así como a diversas complicaciones en el algoritmo de linkage.

#### NOTA: este anáisis es estático de la prueba en cuestión. No coincide siempre el análisis ya que google varía las duraciones e incluso las direcciones sin causa aparente en algunos casos.

# FIN DEL MODELO 1

Mantengo código inservible pero de prueba inicial de clusterización con los algoritmos de las librerias existentes

# PROCESO DE CLUSTERIZACIÓN (no del modelo)

from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from scipy.cluster.hierarchy import cophenet
from pylab import rcParams
import seaborn as sb

np.set_printoptions(precision=4, suppress=True)
#para que no nos salga la notación cientifca y nos salga 4 decimales
plt.style.use('seaborn-whitegrid')

para ver posibles outlier que no cumplan una relación distancia-tiempo adecuada
plt.scatter(ddm['distancia'], ddm['duration'], s=15);

ddm_cluster=ddm.pivot_table(index='origen', columns='destino',values='duration')
ddm_cluster

### PASAR UNA ARRAY O UN DATAFRAME?

Podría convertir el tipo dataframe anterior a tipo array:
ddm_cluster = ddm_cluster.values()
El resultado de la clusterización Z matrix) es el mismo por lo que sigo pasandole un dataframe

ddm_cluster_array = ddm_cluster.values
ddm_cluster_array

ddm_cluster_array[2, 0]
# con la array puedo seleccionar el valor. Posible uso para la clusterización

### USAMOS OTRA LIBRERIA PARA PROBAR: AgglomerativeClustering

from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import 

#### define the model
cluster = AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage='ward',connectivity= ddm_cluster) #### connectivity 

#### fit data and predict 
clusters = cluster.fit_predict(ddm_cluster)
plt.plot(ddm_cluster, clusters)

Z = linkage(ddm_cluster,'single')

plt.figure(figsize=(14, 7))

c = dendrogram(Z, truncate_mode = 'lastp', p=12, leaf_rotation=-25.,leaf_font_size=15., show_contracted=True)

plt.title('Truncated Hierarchical Clustering Dendogram')
plt.xlabel('delivery points')
plt.ylabel('time-distance')


plt.axhline(y=900)

plt.show()

#plt.savefig('plot_dendrogram.png') . Para guardad el dendograma

#pruebo una DEFINICIÓN que saqué de https://stackoverflow.com/questions/11917779/
#how-to-plot-and-annotate-hierarchical-clustering-dendrograms-in-scipy-matplotlib

def augmented_dendrogram(*args, **kwargs):

    ddata = dendrogram(*args, **kwargs)

    if not kwargs.get('no_plot', False):
        for i, d in zip(ddata['icoord'], ddata['dcoord']):
            x = 0.5 * sum(i[1:3])
            y = d[1]
            plt.plot(x, y, 'ro')
            plt.annotate("%.3g" % y, (x, y), xytext=(0, -8),
                         textcoords='offset points',
                         va='top', ha='center')

    return ddata

#pruebo otra DEFINICIÓN: https://stackoverrun.com/es/q/4593125

def plot_tree(P, pos=None): 
    icoord = np.array(P['icoord']) 
    dcoord = np.array(P['dcoord']) 
    color_list = np.array(P['color_list']) 
    xmin, xmax = icoord.min(), icoord.max() 
    ymin, ymax = dcoord.min(), dcoord.max() 
    if pos: 
        icoord = icoord[pos] 
        dcoord = dcoord[pos] 
        color_list = color_list[pos] 
    for xs, ys, color in zip(icoord, dcoord, color_list): 
        plt.plot(xs, ys, color) 
    plt.xlim(xmin-10, xmax + 0.1*abs(xmax)) 
    plt.ylim(ymin, ymax + 0.1*abs(ymax)) 
    plt.show() 

from sklearn.cluster import AgglomerativeClustering

#define the model
cluster = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='ward')  

#fit data and predict 
clusters = cluster.fit_predict(ddm_cluster)
clusters